# Intake Agent related experiments

### Intake prompt

In [1]:
# Force reload modules to pick up changes
import sys
import importlib

# Reload the modules that changed
if 'repoai.llm.pydantic_ai_adapter' in sys.modules:
    importlib.reload(sys.modules['repoai.llm.pydantic_ai_adapter'])
if 'repoai.agents.intake_agent' in sys.modules:
    importlib.reload(sys.modules['repoai.agents.intake_agent'])
if 'repoai.config.settings' in sys.modules:
    importlib.reload(sys.modules['repoai.config.settings'])

print("✓ Modules reloaded")

✓ Modules reloaded


In [2]:
import os
import sys

# Add src to path
sys.path.append(os.path.abspath("../src"))

# Load environment variables
from dotenv import load_dotenv
load_dotenv('/home/timmy/RepoAI_AI/.env', override=True)

# Enable autoreload
%load_ext autoreload
%autoreload 2

print("✓ Environment configured")

✓ Environment configured


In [3]:
from repoai.agents.prompts import INTAKE_INSTRUCTIONS, INTAKE_JAVA_EXAMPLES, INTAKE_SYSTEM_PROMPT

In [4]:
instructions = INTAKE_INSTRUCTIONS
print(instructions)

**How to Parse User Requests:**

1. **Identify Keywords**: Look for action words (add, migrate, refactor, optimize, modernize)
2. **Technology Detection**: Recognize frameworks (Spring, Hibernate, JPA), patterns (JWT, OAuth, MVC)
3. **Scope Indicators**: 
   - File patterns: "auth module", "user service", "all controllers"
   - Package names: "com.example.auth", "authentication package"
   - Specific classes: "UserService", "AuthController"
4. **Constraint Keywords**: "backward compatible", "no breaking changes", "maintain API", "gradual migration"

**Java Build System Detection:**
- Look for mentions of Maven (pom.xml) or Gradle (build.gradle)
- Default to Maven if not specified (most common in enterprise Java)

**Examples of Intent Extraction:**
- "Add JWT authentication" → intent: "add_jwt_authentication"
- "Migrate to Spring Boot 3" → intent: "migrate_spring_boot_3"
- "Refactor user service to use DTOs" → intent: "refactor_user_service_add_dtos"
- "Optimize database queries in orde

In [5]:
system_prompt = INTAKE_SYSTEM_PROMPT
print(system_prompt)

You are an expert Java software architect and refactoring specialist.

Your role is to analyze user requests for Java code refactoring and extract structured information.

**Your Responsibilities:**
1. **Understand Intent**: Identify what the user wants to accomplish (e.g., add JWT authentication, migrate to Spring Boot 3, optimize database queries)
2. **Determine Scope**: Identify which Java packages, classes, and files need to be refactored
3. **Extract Requirements**: List specific technical requirements to fulfill
4. **Identify Constraints**: Note any limitations or requirements (backward compatibility, no breaking changes, etc.)
5. **Consider Java Ecosystem**: Understand Maven/Gradle dependencies, Spring Framework conventions, Jakarta EE standards

**Java-Specific Considerations:**
- Package naming conventions (e.g., com.example.auth, com.example.service)
- Spring Framework annotations (@Service, @RestController, @Autowired, etc.)
- Maven/Gradle build systems and dependency manage

In [6]:
java_examples = INTAKE_JAVA_EXAMPLES
print(java_examples)

**Example 1: Add JWT Authentication**

User Request: "Add JWT authentication to the user service. Use Spring Security and maintain backward compatibility with existing session-based auth."

Expected JobSpec:
```json
{
  "job_id": "job_20250125_143022",
  "intent": "add_jwt_authentication",
  "scope": {
    "target_files": ["src/main/java/com/example/auth/**/*.java", "src/main/java/com/example/security/**/*.java"],
    "target_packages": ["com.example.auth", "com.example.security"],
    "language": "java",
    "build_system": "maven",
    "exclude_patterns": ["**/*Test.java", "**/target/**"]
  },
  "requirements": [
    "Implement JWT token generation using jjwt library",
    "Create JwtService class with token generation and validation methods",
    "Add JWT authentication filter extending OncePerRequestFilter",
    "Configure Spring Security to support JWT authentication",
    "Add refresh token mechanism",
    "Update SecurityConfig to enable JWT auth alongside session auth"
  ],
  "

---

## 🎉 Clean Test: Gemini-Only System

The system has been fully updated to use Gemini exclusively. AIMLAPI support has been removed.

**Restart the kernel and run the cells below.**

In [7]:
# 1. Setup environment
import os
import sys
sys.path.append(os.path.abspath("../src"))

from dotenv import load_dotenv
load_dotenv('/home/timmy/RepoAI_AI/.env', override=True)

print("✓ Environment setup complete")

✓ Environment setup complete


In [8]:
# 1a. Force reload all model modules to pick up Pydantic v2 changes
import sys
import importlib

# List of all modules that need reloading
modules_to_reload = [
    'repoai.models.job_spec',
    'repoai.models.refactor_plan',
    'repoai.models.validation_result',
    'repoai.models.code_changes',
    'repoai.models.PR_description',
    'repoai.models',
    'repoai.config.settings',
    'repoai.llm.pydantic_ai_adapter',
    'repoai.agents.intake_agent',
]

for module_name in modules_to_reload:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
        print(f"✓ Reloaded {module_name}")

print("\n✓ All modules reloaded with Pydantic v2 changes")

✓ Reloaded repoai.models.job_spec
✓ Reloaded repoai.models.refactor_plan
✓ Reloaded repoai.models.code_changes
✓ Reloaded repoai.models.PR_description
✓ Reloaded repoai.models
✓ Reloaded repoai.config.settings
✓ Reloaded repoai.llm.pydantic_ai_adapter
✓ Reloaded repoai.agents.intake_agent

✓ All modules reloaded with Pydantic v2 changes


In [9]:
# 2. Import and verify Gemini setup
from repoai.llm import PydanticAIAdapter, ModelRole
from repoai.config.settings import get_settings
from repoai.agents.intake_agent import run_intake_agent
from repoai.dependencies import IntakeDependencies

# Verify settings
settings = get_settings()
print(f"✓ Google API Key: {'SET' if settings.GOOGLE_API_KEY else 'NOT SET'}")

# Create adapter
adapter = PydanticAIAdapter()

# Check models configured
model_ids = adapter.get_model_ids_with_fallback(ModelRole.INTAKE)
print(f"✓ INTAKE models: {model_ids}")

# Verify model type
model = adapter.get_model(ModelRole.INTAKE)
print(f"✓ Model type: {type(model).__name__}")
print(f"✓ Model: {model}")

✓ Google API Key: SET
✓ INTAKE models: ['gemini-2.0-flash-thinking-exp-01-21', 'gemini-2.0-flash-exp', 'gemini-1.5-flash-002']
✓ Model type: GoogleModel
✓ Model: GoogleModel()


In [10]:
# 3. Create dependencies
deps = IntakeDependencies(
    user_id="timmy_123",
    session_id="session_456",
    repository_url="https://github.com/Khant-Wai-Lin/repo-ai-backend",
)
print(f"✓ Dependencies created: {deps}")

✓ Dependencies created: IntakeDependencies(user_id='timmy_123', session_id='session_456', repository_url='https://github.com/Khant-Wai-Lin/repo-ai-backend', max_retries=3, timeout_seconds=60)


In [18]:
# 4. Test Intake Agent with Gemini + ALL Tools
print("🚀 Running Intake Agent with Gemini...")
print("=" * 60)

try:
    job_spec, meta = await run_intake_agent(
        "Add JWT authentication to the user service and protect /api/*",
        deps,
        adapter=adapter,
    )
    
    print("\n🎉 SUCCESS with Gemini + Tools!\n")
    print("=" * 60)
    print(f"📋 Job ID: {job_spec.job_id}")
    print(f"🎯 Intent: {job_spec.intent}")
    print(f"☕ Language: {job_spec.scope.language}")
    print(f"🔨 Build System: {job_spec.scope.build_system}")
    
    print(f"\n📦 Target Packages ({len(job_spec.scope.target_packages)}):")
    for pkg in job_spec.scope.target_packages:
        print(f"  • {pkg}")
    
    print(f"\n📁 Target Files ({len(job_spec.scope.target_files)}):")
    for pattern in job_spec.scope.target_files[:3]:
        print(f"  • {pattern}")
    if len(job_spec.scope.target_files) > 3:
        print(f"  ... and {len(job_spec.scope.target_files) - 3} more")
    
    print(f"\n✅ Requirements ({len(job_spec.requirements)}):")
    for i, req in enumerate(job_spec.requirements[:5], 1):
        print(f"  {i}. {req}")
    if len(job_spec.requirements) > 5:
        print(f"  ... and {len(job_spec.requirements) - 5} more")
    
    if job_spec.constraints:
        print(f"\n⚠️  Constraints:")
        for constraint in job_spec.constraints:
            print(f"  • {constraint}")
    
    print(f"\n📊 Metadata:")
    print(f"  Model: {meta.model_used}")
    print(f"  Duration: {meta.execution_time_ms:.0f}ms")
    
    print("\n" + "=" * 60)
    print("✨ Intake Agent working perfectly with Gemini!")
    
except Exception as e:
    print(f"\n❌ Error: {type(e).__name__}")
    print(f"Message: {str(e)}")
    import traceback
    traceback.print_exc()

🚀 Running Intake Agent with Gemini...

🎉 SUCCESS with Gemini + Tools!

📋 Job ID: job_20251026_001210_e3277b0a
🎯 Intent: add_jwt_authentication
☕ Language: java
🔨 Build System: maven

📦 Target Packages (3):
  • com.example.user
  • com.example.auth
  • com.example.security

📁 Target Files (3):
  • src/main/java/**/user/**/*.java
  • src/main/java/**/auth/**/*.java
  • src/main/java/**/security/**/*.java

✅ Requirements (8):
  1. Implement JWT token generation (e.g., using jjwt library)
  2. Implement JWT token validation
  3. Create a JwtService class for JWT operations
  4. Develop a custom JWT authentication filter for Spring Security
  5. Configure Spring Security to integrate the JWT filter
  ... and 3 more

⚠️  Constraints:
  • Ensure existing user service functionality remains intact
  • No breaking changes to existing user service API contracts
  • All existing unit and integration tests must continue to pass
  • Add required Maven dependencies to pom.xml

📊 Metadata:
  Model: ge

In [19]:
# Test the full intake agent with Gemini and all tools
try:
    job_spec, meta = await run_intake_agent(
        "Add JWT authentication to the user service and protect /api/*",
        deps,
        adapter=adapter,
    )
    print("🎉 SUCCESS with Gemini + Tools!\n")
    print(f"Job ID: {job_spec.job_id}")
    print(f"Intent: {job_spec.intent}")
    print(f"Language: {job_spec.scope.language}")
    print(f"Build System: {job_spec.scope.build_system}")
    print(f"\nTarget Packages: {job_spec.scope.target_packages}")
    print(f"Target Files: {job_spec.scope.target_files[:2]}...")  # First 2 patterns
    print(f"\nRequirements ({len(job_spec.requirements)} total):")
    for i, req in enumerate(job_spec.requirements[:3], 1):
        print(f"  {i}. {req}")
    print(f"\nConstraints: {job_spec.constraints}")
    print(f"\nModel Used: {meta.model_used}")
    print(f"Duration: {meta.execution_time_ms:.0f}ms")
except Exception as e:
    print(f"❌ Error: {type(e).__name__}")
    print(f"Message: {str(e)}")
    import traceback
    traceback.print_exc()

🎉 SUCCESS with Gemini + Tools!

Job ID: job_20251026_001308_40e64375
Intent: add_jwt_authentication
Language: java
Build System: maven

Target Packages: ['com.example.user', 'com.example.auth', 'com.example.security']
Target Files: ['src/main/java/com/example/user/**/*.java', 'src/main/java/com/example/auth/**/*.java']...

Requirements (7 total):
  1. Implement JWT token generation using a library like jjwt
  2. Create a JwtService for handling token creation, parsing, and validation
  3. Develop a custom JwtAuthenticationFilter to intercept requests and validate JWTs

Constraints: ['Maintain backward compatibility with any existing authentication mechanisms (if applicable)', 'No breaking changes to existing user service API endpoints', 'Ensure all existing unit and integration tests continue to pass after refactoring', 'The solution should be compatible with the current Spring Boot version in use']

Model Used: gemini-2.0-flash-thinking-exp-01-21
Duration: 15638ms
